# Chapter 34: Content Management

Modern applications require dynamic content that can be updated without code deployments. Headless Content Management Systems (CMS) provide the ideal architecture for Next.js applications, enabling content teams to iterate independently while developers maintain full control over presentation layers. Whether using MDX for developer-authored content or integrating with enterprise CMS platforms, Next.js offers powerful patterns for content-driven architectures.

By the end of this chapter, you'll master integrating headless CMS platforms with Next.js, implementing MDX for content-driven pages, configuring real-time preview modes for draft content, optimizing content caching strategies, handling rich text transformations, and building multi-tenant content architectures.

## 34.1 MDX Integration for Content-Driven Pages

MDX combines Markdown with JSX components, enabling interactive documentation and blog posts with full React component support.

### MDX Configuration

```typescript
// next.config.js
const withMDX = require('@next/mdx')({
  extension: /\.mdx?$/,
  options: {
    remarkPlugins: [require('remark-gfm'), require('remark-frontmatter')],
    rehypePlugins: [require('rehype-prism-plus'), require('rehype-slug')],
    providerImportSource: '@mdx-js/react',
  },
});

module.exports = withMDX({
  pageExtensions: ['ts', 'tsx', 'js', 'jsx', 'md', 'mdx'],
});
```

### MDX Components Provider

```typescript
// components/mdx/mdx-components.tsx
import Image from 'next/image';
import Link from 'next/link';
import { CodeBlock } from './code-block';
import { Callout } from './callout';
import { YouTube } from './youtube-embed';

export const mdxComponents = {
  // Override default elements
  h1: ({ children }: { children: React.ReactNode }) => (
    <h1 className="text-4xl font-bold mt-8 mb-4 text-gray-900">{children}</h1>
  ),
  h2: ({ children }: { children: React.ReactNode }) => (
    <h2 className="text-3xl font-semibold mt-8 mb-3 text-gray-800" id={slugify(children)}>
      {children}
    </h2>
  ),
  p: ({ children }: { children: React.ReactNode }) => (
    <p className="text-lg leading-relaxed mb-4 text-gray-700">{children}</p>
  ),
  code: ({ children, className }: { children: string; className?: string }) => {
    if (className?.includes('language-')) {
      return <CodeBlock language={className.replace('language-', '')} code={children} />;
    }
    return <code className="bg-gray-100 px-1.5 py-0.5 rounded text-sm font-mono text-red-600">{children}</code>;
  },
  a: ({ href, children }: { href: string; children: React.ReactNode }) => {
    const isExternal = href?.startsWith('http');
    if (isExternal) {
      return (
        <a 
          href={href} 
          target="_blank" 
          rel="noopener noreferrer" 
          className="text-blue-600 hover:underline"
        >
          {children}
        </a>
      );
    }
    return <Link href={href} className="text-blue-600 hover:underline">{children}</Link>;
  },
  img: ({ src, alt }: { src: string; alt: string }) => (
    <Image 
      src={src} 
      alt={alt} 
      width={800} 
      height={400} 
      className="rounded-lg my-6"
      priority={false}
    />
  ),
  // Custom components
  Callout,
  YouTube,
  Step: ({ number, title, children }: { number: number; title: string; children: React.ReactNode }) => (
    <div className="mb-8 border-l-4 border-blue-500 pl-4">
      <div className="flex items-center gap-2 mb-2">
        <span className="flex items-center justify-center w-6 h-6 rounded-full bg-blue-500 text-white text-sm font-bold">
          {number}
        </span>
        <h3 className="text-xl font-semibold">{title}</h3>
      </div>
      <div className="text-gray-700">{children}</div>
    </div>
  ),
};

function slugify(children: any): string {
  return String(children).toLowerCase().replace(/\s+/g, '-');
}
```

### Dynamic MDX Loading

```typescript
// lib/mdx/loader.ts
import { compileMDX } from 'next-mdx-remote/rsc';
import { mdxComponents } from '@/components/mdx/mdx-components';
import fs from 'fs/promises';
import path from 'path';
import matter from 'gray-matter';

const contentDirectory = path.join(process.cwd(), 'content');

export async function getDocumentBySlug(slug: string) {
  const realSlug = slug.replace(/\.mdx$/, '');
  const filePath = path.join(contentDirectory, `${realSlug}.mdx`);
  
  try {
    const fileContent = await fs.readFile(filePath, 'utf8');
    const { data: frontmatter, content } = matter(fileContent);
    
    const { content: mdxContent } = await compileMDX({
      source: content,
      components: mdxComponents,
      options: {
        parseFrontmatter: false,
      },
    });

    return {
      slug: realSlug,
      frontmatter: frontmatter as {
        title: string;
        description: string;
        publishedAt: string;
        tags: string[];
        author: string;
      },
      content: mdxContent,
    };
  } catch (error) {
    return null;
  }
}

export async function getAllDocuments() {
  const files = await fs.readdir(contentDirectory);
  const mdxFiles = files.filter((file) => file.endsWith('.mdx'));
  
  const documents = await Promise.all(
    mdxFiles.map(async (file) => {
      const slug = file.replace(/\.mdx$/, '');
      const doc = await getDocumentBySlug(slug);
      return {
        slug,
        ...doc?.frontmatter,
      };
    })
  );

  return documents.sort((a, b) => 
    new Date(b.publishedAt).getTime() - new Date(a.publishedAt).getTime()
  );
}
```

```typescript
// app/blog/[slug]/page.tsx
import { notFound } from 'next/navigation';
import { getDocumentBySlug, getAllDocuments } from '@/lib/mdx/loader';
import { Metadata } from 'next';

export async function generateStaticParams() {
  const docs = await getAllDocuments();
  return docs.map((doc) => ({ slug: doc.slug }));
}

export async function generateMetadata({ params }: { params: { slug: string } }): Promise<Metadata> {
  const doc = await getDocumentBySlug(params.slug);
  if (!doc) return {};
  
  return {
    title: doc.frontmatter.title,
    description: doc.frontmatter.description,
  };
}

export default async function BlogPost({ params }: { params: { slug: string } }) {
  const doc = await getDocumentBySlug(params.slug);
  
  if (!doc) {
    notFound();
  }

  return (
    <article className="max-w-3xl mx-auto py-12 px-4">
      <header className="mb-8">
        <h1 className="text-4xl font-bold mb-4">{doc.frontmatter.title}</h1>
        <div className="flex items-center gap-4 text-gray-600">
          <time dateTime={doc.frontmatter.publishedAt}>
            {new Date(doc.frontmatter.publishedAt).toLocaleDateString('en-US', {
              year: 'numeric',
              month: 'long',
              day: 'numeric',
            })}
          </time>
          <span>•</span>
          <span>{doc.frontmatter.author}</span>
        </div>
        <div className="flex gap-2 mt-4">
          {doc.frontmatter.tags.map((tag) => (
            <span key={tag} className="px-2 py-1 bg-gray-100 rounded-full text-sm">
              {tag}
            </span>
          ))}
        </div>
      </header>
      
      <div className="prose prose-lg max-w-none">
        {doc.content}
      </div>
    </article>
  );
}
```

## 34.2 Headless CMS Integration

Integrate with enterprise headless CMS platforms for real-time content updates and collaborative editing.

### Sanity CMS Integration

```typescript
// lib/sanity/client.ts
import { createClient } from 'next-sanity';
import imageUrlBuilder from '@sanity/image-url';

export const client = createClient({
  projectId: process.env.NEXT_PUBLIC_SANITY_PROJECT_ID,
  dataset: process.env.NEXT_PUBLIC_SANITY_DATASET,
  apiVersion: '2024-01-01',
  useCdn: process.env.NODE_ENV === 'production',
  perspective: 'published',
});

// Image URL builder with optimizations
const builder = imageUrlBuilder(client);

export function urlFor(source: any) {
  return builder.image(source).auto('format').fit('max');
}

// Type-safe GROQ queries
export async function getPosts(limit = 10) {
  const query = `*[_type == "post" && defined(slug.current)] | order(publishedAt desc) [0...$limit] {
    _id,
    title,
    slug,
    excerpt,
    publishedAt,
    "author": author->{name, image},
    mainImage,
    "categories": categories[]->title
  }`;
  
  return await client.fetch(query, { limit });
}

export async function getPost(slug: string) {
  const query = `*[_type == "post" && slug.current == $slug][0] {
    _id,
    title,
    slug,
    body,
    publishedAt,
    "author": author->{name, image, bio},
    mainImage,
    "categories": categories[]->title,
    "related": *[_type == "post" && _id != ^._id && count(categories[@._ref in ^.categories[]._ref]) > 0] | order(publishedAt desc) [0...3] {
      title,
      slug,
      excerpt
    }
  }`;
  
  return await client.fetch(query, { slug });
}
```

### Contentful Integration

```typescript
// lib/contentful/client.ts
import { createClient, Entry } from 'contentful';

const client = createClient({
  space: process.env.CONTENTFUL_SPACE_ID!,
  accessToken: process.env.CONTENTFUL_ACCESS_TOKEN!,
  environment: process.env.CONTENTFUL_ENVIRONMENT || 'master',
});

export interface BlogPost {
  title: string;
  slug: string;
  content: any; // Rich text document
  excerpt: string;
  featuredImage: any;
  publishDate: string;
  tags: string[];
  author: Entry<any>;
}

export async function getAllPosts(): Promise<BlogPost[]> {
  const response = await client.getEntries<BlogPost>({
    content_type: 'blogPost',
    order: ['-fields.publishDate'],
    include: 1, // Resolve linked entries (author)
  });
  
  return response.items.map(item => ({
    ...item.fields,
    sys: item.sys,
  })) as BlogPost[];
}

// Rich text renderer
import { documentToReactComponents } from '@contentful/rich-text-react-renderer';
import { BLOCKS, INLINES } from '@contentful/rich-text-types';

const renderOptions = {
  renderNode: {
    [BLOCKS.PARAGRAPH]: (node: any, children: any) => <p className="mb-4">{children}</p>,
    [BLOCKS.HEADING_2]: (node: any, children: any) => <h2 className="text-2xl font-bold mt-8 mb-4">{children}</h2>,
    [BLOCKS.EMBEDDED_ASSET]: (node: any) => {
      const { file, title } = node.data.target.fields;
      return (
        <Image 
          src={`https:${file.url}`} 
          alt={title} 
          width={800} 
          height={600} 
          className="rounded-lg my-6"
        />
      );
    },
    [INLINES.HYPERLINK]: (node: any, children: any) => (
      <a href={node.data.uri} className="text-blue-600 underline" target="_blank" rel="noopener noreferrer">
        {children}
      </a>
    ),
  },
};

export function renderRichText(document: any) {
  return documentToReactComponents(document, renderOptions);
}
```

## 34.3 Preview Mode and Draft Content

Implement real-time preview functionality for content editors to view unpublished changes.

### Draft Mode API

```typescript
// app/api/preview/route.ts
import { draftMode } from 'next/headers';
import { redirect } from 'next/navigation';
import { client } from '@/lib/sanity/client';

export async function GET(request: Request) {
  const { searchParams } = new URL(request.url);
  const secret = searchParams.get('secret');
  const slug = searchParams.get('slug');
  const type = searchParams.get('type') || 'post';

  // Verify secret
  if (secret !== process.env.CONTENT_PREVIEW_SECRET) {
    return new Response('Invalid token', { status: 401 });
  }

  // Verify the slug exists in draft
  const query = `*[_type == $type && slug.current == $slug][0]{
    slug,
    _id,
    _updatedAt
  }`;
  
  const doc = await client.fetch(query, { type, slug });
  
  if (!doc) {
    return new Response('Document not found', { status: 404 });
  }

  // Enable draft mode
  const draft = await draftMode();
  draft.enable();

  // Set cookie for Sanity preview
  const cookieStore = await cookies();
  cookieStore.set('__sanity_preview', JSON.stringify({
    id: doc._id,
    updatedAt: doc._updatedAt,
  }));

  redirect(`/${type === 'post' ? 'blog' : 'page'}/${slug}`);
}

// app/api/preview/disable/route.ts
import { draftMode } from 'next/headers';
import { redirect } from 'next/navigation';

export async function GET() {
  const draft = await draftMode();
  draft.disable();
  redirect('/');
}
```

### Preview Client Configuration

```typescript
// lib/sanity/preview-client.ts
import { createClient } from 'next-sanity';

export const previewClient = createClient({
  projectId: process.env.NEXT_PUBLIC_SANITY_PROJECT_ID,
  dataset: process.env.NEXT_PUBLIC_SANITY_DATASET,
  apiVersion: '2024-01-01',
  useCdn: false,
  perspective: 'previewDrafts', // Fetch drafts
  token: process.env.SANITY_API_READ_TOKEN, // Requires token for drafts
});

export const getPreviewPost = async (slug: string) => {
  const query = `*[_type == "post" && slug.current == $slug] | order(_updatedAt desc) [0] {
    _id,
    title,
    "slug": slug.current,
    body,
    _updatedAt
  }`;
  
  return await previewClient.fetch(query, { slug });
};
```

### Preview Indicator Component

```typescript
// components/preview-indicator.tsx
import { draftMode } from 'next/headers';
import Link from 'next/link';

export async function PreviewIndicator() {
  const { isEnabled } = await draftMode();

  if (!isEnabled) return null;

  return (
    <div className="fixed bottom-4 right-4 z-50 bg-yellow-400 text-black px-4 py-2 rounded-lg shadow-lg flex items-center gap-4">
      <div className="flex items-center gap-2">
        <span className="relative flex h-3 w-3">
          <span className="animate-ping absolute inline-flex h-full w-full rounded-full bg-red-400 opacity-75"></span>
          <span className="relative inline-flex rounded-full h-3 w-3 bg-red-500"></span>
        </span>
        <span className="font-medium">Preview Mode</span>
      </div>
      <Link 
        href="/api/preview/disable" 
        className="text-sm underline hover:no-underline"
      >
        Exit Preview
      </Link>
    </div>
  );
}
```

```typescript
// app/layout.tsx
import { PreviewIndicator } from '@/components/preview-indicator';

export default function RootLayout({ children }: { children: React.ReactNode }) {
  return (
    <html lang="en">
      <body>
        {children}
        <PreviewIndicator />
      </body>
    </html>
  );
}
```

## 34.4 Content Caching and Revalidation

Optimize content delivery with strategic caching and on-demand revalidation.

### ISR with CMS Webhooks

```typescript
// app/api/webhooks/sanity/route.ts
import { revalidateTag } from 'next/cache';
import { NextRequest } from 'next/server';
import { isValidSignature, SIGNATURE_HEADER_NAME } from '@sanity/webhook';

const secret = process.env.SANITY_WEBHOOK_SECRET;

export async function POST(req: NextRequest) {
  const signature = req.headers.get(SIGNATURE_HEADER_NAME);
  const body = await req.text();
  
  if (!signature || !isValidSignature(body, signature, secret!)) {
    return new Response('Invalid signature', { status: 401 });
  }

  const json = JSON.parse(body);
  const { _type, slug } = json;

  // Revalidate specific paths based on content type
  if (_type === 'post') {
    await revalidateTag('posts');
    await revalidateTag(`post-${slug.current}`);
    
    // Revalidate the specific page
    try {
      await fetch(`${process.env.NEXT_PUBLIC_URL}/api/revalidate`, {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify({ path: `/blog/${slug.current}` }),
      });
    } catch (error) {
      console.error('Revalidation error:', error);
    }
  }

  return Response.json({ revalidated: true, now: Date.now() });
}
```

### Cache-Tags Strategy

```typescript
// lib/cms/cache-tags.ts
export const CACHE_TAGS = {
  posts: 'posts',
  post: (slug: string) => `post-${slug}`,
  authors: 'authors',
  settings: 'settings',
};

// Usage in data fetching
import { unstable_cache } from 'next/cache';

export const getCachedPost = unstable_cache(
  async (slug: string) => {
    return await getPost(slug);
  },
  ['post-detail'],
  {
    tags: [CACHE_TAGS.posts, CACHE_TAGS.post(slug)],
    revalidate: 60, // 60 seconds
  }
);
```

## Key Takeaways from Chapter 34

1. **MDX Integration**: Use `@next/mdx` for compiling Markdown with JSX at build time, or `next-mdx-remote` for dynamic content loading. Create a component mapping system to standardize styling and inject custom React components like callouts, code blocks, or interactive demos into content.

2. **Headless CMS Patterns**: Configure CMS clients (Sanity, Contentful, Strapi) with environment-specific settings (`useCdn: true` for production, `false` for drafts). Use GROQ (Sanity) or GraphQL (Contentful) to fetch precisely the data needed, leveraging linked entry resolution to minimize API calls.

3. **Preview Mode**: Implement Draft Mode API routes to authenticate editors and enable preview perspectives. Use `draftMode().enable()` to set cookies that bypass static generation, allowing real-time viewing of unpublished content. Always verify webhook signatures to prevent unauthorized preview access.

4. **Rich Text Handling**: Transform CMS-specific rich text formats (Portable Text, Rich Text JSON) into React components using renderer libraries. Handle embedded assets (images, videos) by resolving references and applying Next.js Image optimization.

5. **Revalidation Strategy**: Configure CMS webhooks to trigger `revalidateTag()` or `revalidatePath()` when content updates. Use cache tags to granularly invalidate content without rebuilding entire pages. Implement fallback: 'blocking' for ISR to generate missing pages on-demand.

6. **Type Safety**: Generate TypeScript types from CMS schemas (Sanity TypeGen, Contentful Content Type generator) to ensure compile-time checking of content structures. Use Zod validation for runtime safety when consuming external CMS data.

7. **Asset Optimization**: Use CMS image URL builders (Sanity's `image-url`, Contentful's `cf.url`) to apply transformations (format conversion, resizing) at the CDN level, reducing bandwidth and improving Core Web Vitals.

## Coming Up Next

**Chapter 35: E-commerce Integration**

With content management mastered, it's time to handle transactions. In Chapter 35, we'll explore integrating Shopify and Stripe for product catalogs, shopping carts, and checkout flows. You'll learn how to implement secure payment processing, manage inventory synchronization, handle webhooks for order fulfillment, and optimize e-commerce performance for conversion rates.